In [11]:
import pandas as pd
import pickle

In [12]:
with open('./data/cleaned_data/players_advanced_stats.pickle', 'rb') as adv_stats:
    players_advanced_stats = pickle.load(adv_stats)

In [13]:
with open('./data/cleaned_data/players_stats.pickle', 'rb') as play_stats:
    players_stats = pickle.load(play_stats)

In [15]:
players_stats[2001]

{'playoffs':               Player  Age   Tm   G  GS    MP         ID
 0         Ray Allen*   25  MIL  18  18  42.7  allenra02
 1       Rafer Alston   24  MIL   5   0   1.6  alstora01
 2       John Amaechi   30  ORL   4   0   7.3  amaecjo01
 3     Derek Anderson   26  SAS   7   7  27.7  anderde01
 4       Greg Anthony   33  POR   2   0   8.5  anthogr01
 ..               ...  ...  ...  ..  ..   ...        ...
 185   Jason Williams   25  SAC   8   8  23.9  willija02
 186  Jerome Williams   27  TOR  11   0  14.9  willije01
 187   Monty Williams   29  ORL   3   0   4.7  willimo01
 188   Scott Williams   32  MIL  17  17  22.2  willisc01
 189      Wang Zhizhi   23  DAL   5   0   4.6  zhizhwa01
 
 [190 rows x 7 columns],
 'leagues':                   Player  Age   Tm   G  GS    MP         ID
 0     Mahmoud Abdul-Rauf   31  MEM  41   0  11.9  abdulma02
 1      Tariq Abdul-Wahad   26  DEN  29  12  14.5  abdulta01
 2    Shareef Abdur-Rahim   24  MEM  81  81  40.0  abdursh01
 3         Cory Alexan

In [14]:
players_advanced_stats[2001]

{'playoffs':               Player   Tm   G   PER   WS         ID
 0         Ray Allen*  MIL  18  23.7  3.6  allenra02
 1       Rafer Alston  MIL   5 -14.2 -0.1  alstora01
 2       John Amaechi  ORL   4  19.4  0.1  amaecjo01
 3     Derek Anderson  SAS   7   4.3 -0.3  anderde01
 4       Greg Anthony  POR   2   2.6 -0.1  anthogr01
 ..               ...  ...  ..   ...  ...        ...
 185   Jason Williams  SAC   8  10.4  0.2  willija02
 186  Jerome Williams  TOR  11  15.1  0.5  willije01
 187   Monty Williams  ORL   3  28.3  0.1  willimo01
 188   Scott Williams  MIL  17  17.6  1.2  willisc01
 189      Wang Zhizhi  DAL   5  16.3  0.0  zhizhwa01
 
 [190 rows x 6 columns],
 'leagues':                   Player   Tm   G   PER   WS         ID
 0     Mahmoud Abdul-Rauf  MEM  41  16.7  0.7  abdulma02
 1      Tariq Abdul-Wahad  DEN  29   5.8 -0.4  abdulta01
 2    Shareef Abdur-Rahim  MEM  81  19.1  8.5  abdursh01
 3         Cory Alexander  ORL  26   6.3 -0.1  alexaco01
 4     Courtney Alexander  WA

In [ ]:
def player_efficiency(player_id: str, game_types: list, players_stats: pd.DataFrame, players_advanced_stats: pd.DataFrame):
    """Return Player Efficiency Rating for each player"""
    minutes_played = 0
    efficiency = 0
    efficiency_x_minutes = 0
    efficiency_total = 0
    for game_type in game_types:
        try:
            minutes_played = players_stats[game_type].query(f"ID == '{player_id}'")["MP"].iloc[0]
            efficiency = players_advanced_stats[game_type].query(f"ID == '{player_id}'")["PER"].iloc[0]

            efficiency_x_minutes += efficiency * minutes_played
        except:
            pass

    if minutes_played != 0:
        efficiency_total = round((efficiency_x_minutes/minutes_played), 3)
    else:
        efficiency_total = 0

    return efficiency_total

In [ ]:
stats = {}
stats_columns = ["ID", "salary_perc", "last_year_of_contract", "age", "games_played_perc", "games_started_perc", "minutes_played", "avg_minutes_played", "WS48", "PER", "team_successes",
              "defensive", "most_improved", "most_valuable", "most_valuable_finals", "sixth_man", "all_league", "all_def"]

In [ ]:
for year in years:
    player_list = set(players_stats[year]["playoffs"]["ID"].values.tolist() + players_stats[year]["leagues"]["ID"].values.tolist())
    stats[year] = pd.DataFrame(columns=stats_columns)
    stats[year]['ID'] = [player for player in player_list]
    stats[year] = stats[year].set_index("ID")
    stats[year]['last_year_of_contract'] = 'noinfo'
    stats[year] = stats[year].fillna(0)
    for player_id in player_list:
        stats[year].loc[[f'{player_id}'], ['WS48']] = winshares_per48(player_id, game_types, players_stats[year], players_advanced_stats[year])
        stats[year].loc[[f'{player_id}'], ['PER']] = player_efficiency(player_id, game_types, players_stats[year], players_advanced_stats[year])